In [1]:
!pip install numpy 
!pip install pandas
!pip install matplotlib 
!pip install seaborn 
!pip install sklearn
!pip install tensorflow
!pip install keras-tuner


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.svm import SVC
import keras_tuner as kt
from sklearn.linear_model import LogisticRegression

In [3]:
wine_ds = pd.read_csv('WQ_Dataset.csv')

In [4]:
x = wine_ds.drop("quality", axis=1)
y = wine_ds["quality"].apply(lambda y_value: "perfect" if y_value == 10 else ("good" if (y_value >= 7 and y_value < 10) else ("average" if (y_value >= 5 and y_value < 7) else ("bad" if (y_value >= 3 and y_value < 5) else "inedible"))))

In [5]:
y = pd.factorize(y)[0]
num_classes = len(np.unique(y))
y = to_categorical(y, num_classes = num_classes)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

In [7]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [8]:
param_grid_log_reg = {
    'C': [0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}

In [9]:
log_reg = LogisticRegression(random_state=5, multi_class='ovr')

grid_search_log_reg = GridSearchCV(estimator=log_reg, param_grid=param_grid_log_reg, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_log_reg.fit(x_train, np.argmax(y_train, axis=1))
best_log_reg = grid_search_log_reg.best_estimator_

In [10]:
print("Parameters for Logistic Regression:", grid_search_log_reg.best_params_)
print("Score for Logistic Regression:", grid_search_log_reg.best_score_)

Parameters for Logistic Regression: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
Score for Logistic Regression: 0.8365962009803921


In [11]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

In [12]:
rf = RandomForestClassifier(random_state=5)
rf = RandomForestClassifier(random_state=5)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_rf.fit(x_train, np.argmax(y_train, axis=1))
best_rf = grid_search_rf.best_estimator_

In [13]:
print("Parameters for Random Forest:", grid_search_rf.best_params_)
print("Score for Random Forest:", grid_search_rf.best_score_)

Parameters for Random Forest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 100}
Score for Random Forest: 0.8670833333333334


In [14]:
param_grid_gbm = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

In [15]:
gbm = GradientBoostingClassifier(random_state=5)
grid_search_gbm = GridSearchCV(estimator=gbm, param_grid=param_grid_gbm, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_gbm.fit(x_train, np.argmax(y_train, axis=1))
best_gbm = grid_search_gbm.best_estimator_

In [16]:
print("Parameters for GBM:", grid_search_gbm.best_params_)
print("Score for GBM:", grid_search_gbm.best_score_)

Parameters for GBM: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Score for GBM: 0.8717769607843138


In [17]:
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

In [18]:
svm = SVC(probability=True, random_state=5)
grid_search_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_svm.fit(x_train, np.argmax(y_train, axis=1))
best_svm = grid_search_svm.best_estimator_

In [19]:
print("Parameters for SVM:", grid_search_svm.best_params_)
print("Score for SVM:", grid_search_svm.best_score_)

Parameters for SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Score for SVM: 0.8451838235294117


In [20]:
log_reg_train_preds = best_log_reg.predict_proba(x_train)
log_reg_test_preds = best_log_reg.predict_proba(x_test)

In [21]:
rf_train_preds = best_rf.predict_proba(x_train)
gbm_train_preds = best_gbm.predict_proba(x_train)
svm_train_preds = best_svm.predict_proba(x_train)

In [22]:
rf_test_preds = best_rf.predict_proba(x_test)
gbm_test_preds = best_gbm.predict_proba(x_test)
svm_test_preds = best_svm.predict_proba(x_test)

In [23]:
x_train_stack = np.hstack((rf_train_preds, gbm_train_preds, svm_train_preds, log_reg_train_preds))
x_test_stack = np.hstack((rf_test_preds, gbm_test_preds, svm_test_preds, log_reg_test_preds))

In [24]:
def create_meta_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [25]:
meta_nn_model = create_meta_nn_model(x_train_stack.shape[1])
meta_nn_model.fit(x_train_stack, y_train, epochs=50, batch_size=10, verbose=1)



Epoch 1/50


128/128 [==============================] - 1s 2ms/step - loss: 0.7237 - accuracy: 0.9030
Epoch 2/50
128/128 [==============================] - 0s 1ms/step - loss: 0.0963 - accuracy: 0.9742
Epoch 3/50
128/128 [==============================] - 0s 1ms/step - loss: 0.0188 - accuracy: 0.9992
Epoch 4/50
128/128 [==============================] - 0s 1ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 5/50
128/128 [==============================] - 0s 1ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 6/50
128/128 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 7/50
128/128 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 8/50
128/128 [==============================] - 0s 1ms/step - loss: 7.6222e-04 - accuracy: 1.0000
Epoch 9/50
128/128 [==============================] - 0s 1ms/step - loss: 5.6386e-04 - accuracy: 1.0000
Epoch 10/50
128/128 [==============================] - 0s 1ms/step - loss: 4.3786e-04 - 

In [26]:
y_pred_stack = meta_nn_model.predict(x_test_stack)
y_pred_stack = np.argmax(y_pred_stack, axis=1)

10/10 [==============================] - 0s 1ms/step


In [27]:
print(f"Ensemble model accuracy: {accuracy_score(np.argmax(y_test, axis=1), y_pred_stack)}")
print(classification_report(np.argmax(y_test, axis=1), y_pred_stack))

Ensemble model accuracy: 0.884375
              precision    recall  f1-score   support

           0       0.90      0.97      0.93       267
           1       0.80      0.52      0.63        46
           2       0.25      0.14      0.18         7

    accuracy                           0.88       320
   macro avg       0.65      0.54      0.58       320
weighted avg       0.87      0.88      0.87       320

